# Welcome to Jupyter Notebooks on the Intel AI DevCloud! 
This document covers the basics of the Jupyter Notebook access to the Intel AI DevCloud. It is not a tutorial on the Jupyter Notebook itself. Rather, we will run through a few examples of how to use the computational resources available on the DevCloud *beyond* the notebook.

The diagram below illustrates the high-level organization of the DevCloud. This tutorial explains how to navigate this organization. 

<img src="https://access.colfaxresearch.com/images/cluster-jn-organization.svg" style="max-width:600px;" />


## Service Terms

By using the Intel AI DevCloud, you are agreeing to the following Service Terms: <br />
<a href="https://access.colfaxresearch.com/doc/Colfax_Cluster_Service_Terms.pdf">https://access.colfaxresearch.com/doc/Colfax_Cluster_Service_Terms.pdf</a>

# 1. 画像データをダウンロードする

In [ ]:
!wget https://cs298395642e8d6x4498x8b7.blob.core.windows.net/share/data.zip

In [ ]:
!mkdir train_data

In [ ]:
!mv data.zip train_data/

In [ ]:
!unzip train_data/data.zip -d train_data/

# 2. モデルファイルをOpenVINOのIR形式に変換する

事前にモデルファイル「model.pb」をJupyter Notebookにアップロードする必要があります。

In [ ]:
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model model.pb \
--data_type FP32 \
--output_dir . \
--input_shape [1,224,224,3]

# 3. バックエンドにてIRを実行するPythonスクリプトを作成

In [ ]:
%%writefile run_inference.py
from __future__ import print_function
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras import models
import numpy as np
import glob
import random
import time
import os
import cv2
import pandas as pd
from PIL import Image
import PIL
import io
import sys

from openvino.inference_engine import IENetwork, IEPlugin

def inference_openvino(total = 100, target_device="CPU"):
    model_xml = 'model.xml'
    model_bin = 'model.bin'

    # Plugin initialization for specified device and load extensions library if specified
    # Set the desired device name as 'device' parameter. This sample support these 3 names: CPU, GPU, MYRIAD
    ie = IEPlugin(device=target_device, plugin_dirs='')

    # Read IR
    net = IENetwork(model=model_xml, weights=model_bin)

    input_blob = next(iter(net.inputs))
    out_blob = next(iter(net.outputs))
    net.batch_size = 1

    # Loading model to the plugin
    exec_net = ie.load(network=net)
    
    list_df = pd.DataFrame( columns=['正解ラベル','予測ラベル','全処理時間(msec)','推論時間(msec)'] )

    total_spent_time = 0
    total_infer_spent_time = 0
    
    for j in range(total):
        time1 = time.time()
        file_list = glob.glob("train_data/test/*/*")
        img_path = random.choice(file_list)
        img_cat = os.path.split(os.path.dirname(img_path))[1]
        # Read and pre-process input images
        n, c, h, w = net.inputs[input_blob].shape
        images = np.ndarray(shape=(n, c, h, w))
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if image.shape[:-1] != (h, w):
            image = cv2.resize(image, (w, h))
        frame = image
        image = image.transpose((2, 0, 1))  # Change data layout from HWC to CHW
        image = image.reshape((n, c, h, w))
        #image = preprocess_input(image)
        images[0] = image

        # Start sync inference
        time2 = time.time()
        preds = exec_net.infer(inputs={input_blob: images})
        
        infer_spent_time = time.time() - time2
        total_infer_spent_time += infer_spent_time
        
        spent_time = time.time() - time1
        total_spent_time += spent_time
        
        preds = preds[out_blob]
        top = preds[0].argsort()[-1:][::-1]
        #pred_label = labels[top[0]]
        pred_label = top[0]
        tmp_se = pd.Series( [img_cat, pred_label, str(int(spent_time * 1000)), str(int(infer_spent_time * 1000)) ], index=list_df.columns )
        list_df = list_df.append( tmp_se, ignore_index=True ) 

    print()
    print('全' + str(total) + '枚 完了！')
    print()
    print("平均処理時間: " + str(int((total_spent_time / total)*1000.0)) + " ms/枚")
    print("平均推論時間: " + str(int((total_infer_spent_time / total)*1000.0)) + " ms/枚")
    print(list_df)
    
    return int((total_spent_time / total)*1000.0), int((total_infer_spent_time / total)*1000.0)

if __name__ == '__main__':
    cpu_total_time, cpu_infer_time = inference_openvino(total=50, target_device="CPU")
    sys.exit(0)

# 4. バックエンドで動作するShell Scriptを作成

In [ ]:
%%writefile run.sh
cd $PBS_O_WORKDIR
echo "* Hello world from compute server `hostname`!"
echo "* The current directory is ${PWD}."
echo "* Compute server's CPU model and number of logical CPUs:"
lscpu
echo "* Python available to us:"
which python
python --version

#Setup related environment variables
source /opt/intel/openvino/bin/setupvars.sh

#Install related python libraries
pip install networkx defusedxml protobuf intel-tensorflow==1.15.0 pillow --user

#Run python script in parallel
numactl -N 0 -m 0 python object_detection_demo_ssd_async.py &
numactl -N 1 -m 1 python object_detection_demo_ssd_async.py &

wait
echo "*Bye"
# Remember to have an empty line at the end of the file; otherwise the last command will not run


# 5. Job Submit

In [ ]:
#Skylakeを使う場合はこちら
!qsub run.sh -l nodes=1:fpga_compile:ppn=2

#Cascade Lakeを使う場合はこちら
#!qsub run.sh -l nodes=1:experimental:ppn=2

JobのStatusは下記コマンドにて確認

In [ ]:
!qstat

# 6. Jobの結果を確認

標準出力はoから始まる名前のファイル

In [ ]:
%cat run.sh.o*

エラー出力はeから始まる名前のファイル

In [ ]:
%cat run.sh.e*

# 以降はおまけ

In [ ]:
!echo "* How many compute servers are available?"
!pbsnodes | grep "^c" | wc -l

!echo "* How many of them are free?"
!pbsnodes | grep "state = free" | wc -l

!echo "* What are the time limits for queued jobs?"
!qmgr -c 'p q batch' | grep walltime

!echo "* What is the configuration of the available compute servers?"
!pbsnodes | grep properties | sort | uniq